In [2]:
import pandas as pd

# ## IMDB Movies
database = pd.read_csv("IMDb_movies.csv", sep = ",")
database["date_published"] = pd.to_datetime(database["date_published"])
database.head(15)


# ### Limpando a base de dados
# - reviews_from_users 
# - reviews_from_critics
# Queremos excluir da base de dados a parte que não possui nota dos críticos nem dos expectadores e algumas colunas que não farão parte do escopo do projeto.

df1 = database[(database["reviews_from_critics"].notnull()) & (database["reviews_from_users"].notnull())]
df1.reset_index(drop = True, inplace = True)  ## resetando o indice das linhas
df1.drop(columns = ["usa_gross_income", "duration", "writer", "metascore", "reviews_from_critics", "reviews_from_users", "votes", "avg_vote"], axis = "columns", inplace = True) ## excluindo colunas
df1

database2 = database[(database["reviews_from_critics"].isnull()) | (database["reviews_from_users"].isnull())]
A = database.shape[0]
B = database2.shape[0]
print("O número de linhas inicial é:",  database.shape[0])
print("O número de linhas retiradas é:",  database2.shape[0])
print("O número de linhas restante é:",  A - B)

df1.rename(columns={'date_published': 'release_date'}, inplace=True)
df1.shape


# ## IMDB Ratings

df2 = pd.read_csv("IMDb_ratings.csv", sep = ",")
df2.drop(["total_votes", "votes_10", "votes_9", "votes_8",
                "votes_7", "votes_6","votes_5", "votes_4", "votes_3",
                "votes_2", "votes_1", "us_voters_votes", "us_voters_rating", "median_vote",
                "non_us_voters_rating", "non_us_voters_votes", "top1000_voters_rating", "top1000_voters_votes", "weighted_average_vote"], axis = "columns", inplace = True)
df2['votes'] = df2['allgenders_0age_votes'] + df2['allgenders_18age_votes'] + df2['allgenders_30age_votes'] + df2['allgenders_45age_votes']
df2 ['allgenders_allages_votes'] = df2['votes']
df2.rename(columns={'mean_vote':'avg_vote', 'date_published': 'release_date'}, inplace=True)
df2['allgenders_allages_avg_vote'] = df2['avg_vote']
# total_votes já está no IMDb Movies


# ### Juntando as duas base de dados com merge
left = df1
right = df2
df = left.merge(right, on='imdb_title_id')
df.head(15)
df.shape

O número de linhas inicial é: 81273
O número de linhas retiradas é: 14770
O número de linhas restante é: 66503


(66503, 46)

In [3]:
# ### Tirando as linhas com NaN 
df.dropna(inplace=True)


### Acrescentando os dados da Netflix
netflix_only = pd.read_csv('netflix_titles.csv')
netflix_only['on_netflix'] = True
netflix_only = netflix_only[netflix_only['type'] == 'Movie'][['title', 'on_netflix']]
netflix = df.merge(netflix_only, on='title', how='left')
netflix['on_netflix'].fillna(False, inplace=True)

print("Tamanho dos dados:", netflix.shape)

# Exportando pro Excel

netflix.to_csv("base_consolidada.csv", sep = ";", index = False) ## não salva o indice

Tamanho dos dados: (5244, 47)
